In [52]:
import pandas as pd
df=pd.read_csv('kc_house_data.csv')

In [53]:
df['date'] = pd.to_datetime(df.date)


In [54]:
df['bedrooms'] = df[['bedrooms','sqft_living']].apply(lambda x: 3 if (x.sqft_living > 1000)&(x.bedrooms==0) else x.bedrooms, axis=1)


scaling not necessary, transforming, prove below

things to transform: sqft_living, sqft_lot, sqft_above, sqf_basement

best Skew: log for living, box cox for lot, 

Features: living:lot ratio, above to basement ratio, price/sqft, how long ago was renovated, size of house compared to 15, size of lot compared to 15

encode: zip code(look into hashing for this), view, waterfront, 

others added grade and condition, because they are both rankings, something to consider: they were already encoded for us

In [55]:
from scipy.stats import boxcox

In [56]:
import numpy as np

In [57]:
df['log_living']= np.log(df['sqft_living'])
df.log_living.skew()

-0.03543769299569605

In [58]:
df['log_lot']= np.log(df['sqft_lot'])
df.log_lot.skew()

0.9621711661039318

In [59]:
from scipy.stats import boxcox

In [60]:
df['boxcox_lot']= boxcox(df.sqft_lot)[0]
df.boxcox_lot.skew()

-0.10691228776704689

features below

In [61]:
df['price/sqft']= df['price']/df['sqft_living']


In [62]:
df['lot/inside']= df['sqft_lot']/df['sqft_living']


In [63]:
df['base/upper']= df['sqft_basement']/df['sqft_living']


In [64]:
df['sqft_liv/15']= df['sqft_living']/df['sqft_living15']

In [65]:
df['sqft_LOT/15']= df['sqft_lot']/df['sqft_lot15']

In [66]:
from datetime import date

In [67]:
today = date.today() 
df['remodel_age']= df['yr_renovated'].apply(lambda x: today.year - x if (x > 0) else 0)



    

In [70]:
abs(df.corr()['price'].sort_values(ascending=False))[:40]

price            1.000000
sqft_living      0.702035
grade            0.667434
log_living       0.611704
sqft_above       0.605567
sqft_living15    0.585379
price/sqft       0.554703
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.309059
lat              0.307003
sqft_liv/15      0.302192
zip__98004       0.271131
waterfront       0.266369
floors           0.256794
zip__98039       0.212558
zip__98040       0.204873
zip__98112       0.169842
log_lot          0.161318
boxcox_lot       0.156340
base/upper       0.135539
zip__98006       0.133695
yr_renovated     0.126434
zip__98033       0.102556
zip__98105       0.090974
sqft_lot         0.089661
zip__98075       0.088676
zip__98199       0.083659
remodel_age      0.083525
sqft_lot15       0.082447
zip__98119       0.078085
zip__98102       0.068738
zip__98109       0.065846
zip__98005       0.065114
zip__98074       0.057207
yr_built         0.054012
zip__98053       0.051974
zip__98052  

sqft_living is most correlated to price. 

In [71]:
from scipy import stats

In [72]:
linear_regression=stats.linregress(df['sqft_living'], df['price'])
linear_regression

LinregressResult(slope=280.6235678974483, intercept=-43580.74309447408, rvalue=0.7020350546118003, pvalue=0.0, stderr=1.9363985519989133, intercept_stderr=4402.6896903039)

In [73]:
def price_call(sqft_living):
    price=sqft_living * linear_regression.slope + linear_regression.intercept
    return price

In [74]:
df['predicted_price'] = price_call(df['sqft_living'])


In [75]:
import numpy as py

divide by length BEFORE taking sqrt

STEPS: 
find correlation
calculate linear regression linear_regression=stats.linregress(df['sqft_living'], df['price'])

do function :
def price_call(sqft_living):
    price=sqft_living * linear_regression.slope + linear_regression.intercept
    return price

Predict price using above function df['predicted_price'] = price_call(df['sqft_living'])
df

calculate difference between prediction versus actual price 
df['diff'] = df.price-df.predicted_price

square diff
df['square_diff'] = py.square(df['diff'])

divide sum by index find sqrt of result: 
py.sqrt(sum1/len(df.square_diff))


In [76]:
from sklearn.linear_model import LinearRegression

In [77]:
l = LinearRegression()

In [78]:
x = df[['remodel_age','zipcode','log_living', 'grade', 'sqft_above','boxcox_lot', 'sqft_living15', 'bathrooms', 'view', 'sqft_basement','bedrooms', 'sqft_liv/15']]

y = df[['price']]


In [79]:
x2 = pd.merge(x, zipcode, left_index=True, right_index=True)
x2

,remodel_age,zipcode,log_living,grade,sqft_above,boxcox_lot,sqft_living15,bathrooms,view,sqft_basement,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0,98178,7.073270,7,1180,4.203914,1340,1.00,0,0,...,0,0,0,0,0,0,1,0,0,0
1,30,98125,7.851661,7,2170,4.249675,1690,2.25,0,400,...,0,0,0,0,0,0,0,0,0,0
2,0,98028,6.646391,6,770,4.305976,2720,1.00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,98136,7.580700,7,1050,4.180566,1360,3.00,0,910,...,0,0,0,0,0,0,0,0,0,0
4,0,98074,7.426549,8,1680,4.269174,1800,2.00,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0,98103,7.333023,8,1530,3.847818,1530,2.50,0,0,...,0,0,0,0,0,0,0,0,0,0
21609,0,98146,7.745003,8,2310,4.209269,1830,2.50,0,0,...,1,0,0,0,0,0,0,0,0,0
21610,0,98144,6.927558,7,1020,3.892628,1020,0.75,0,0,...,0,0,0,0,0,0,0,0,0,0
21611,0,98027,7.377759,8,1600,4.027035,1410,2.50,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
x2.drop('zipcode', axis=1, inplace=True)

In [81]:
zipcode = pd.get_dummies(x.zipcode)
zipcode

,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21609,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
21610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
l.fit(x2,y)

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [83]:
df['predict_price'] = l.predict(x2)

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [84]:
df['diff2']= df.price-df.predict_price

In [85]:
df['square_diff2'] = py.square(df['diff2'])

In [86]:
sum2=sum(df.square_diff2)
sum2

571935342839391.8

In [87]:
py.sqrt(sum2/len(df.square_diff2))

162673.17404151498

In [88]:
from sklearn.metrics import mean_squared_error

In [89]:
pred = l.predict(x2)

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [90]:
py.sqrt(mean_squared_error(df['price'], pred))

162673.17404151434

In [92]:
from sklearn.model_selection import train_test_split

In [93]:
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=.33, random_state=9)

In [99]:
model=LinearRegression(fit_intercept=True)

In [100]:
model.fit(x_train,y_train['price'])

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [104]:
pred_frame = pd.DataFrame(model.predict(x_test))


/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [106]:
import numpy as np

In [107]:
np.sqrt(mean_squared_error (y_test, pred_frame[0]))

159706.9057899167

In [108]:
from sklearn.linear_model import Lasso, Ridge

In [111]:
lasso_r = Lasso()
lasso_r.fit(x2,y)

/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/hannahgraham/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.315e+13, tolerance: 2.913e+11
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [112]:
(lasso_r.sparse_coef_)

<1x81 sparse matrix of type '<class 'numpy.float64'>'
	with 81 stored elements in Compressed Sparse Row format>

In [113]:
ridge_r = Ridge()
ridge_r.fit(x,y)

Ridge()

In [115]:
ridge_r.coef_

array([[ 3.64842091e+03,  2.41373927e+02, -5.74735731e+05,
         1.00751937e+05,  3.91728640e+02, -1.62412547e+05,
         5.24319658e+01, -9.08351776e+03,  7.59477931e+04,
         4.43855541e+02, -3.71672399e+03,  2.55048854e+04]])

In [117]:
elasticnet_r= ElasticNet(l1_ratio=1, alpha=1000)
elasticnet_r.fit(x,y)

NameError: name 'ElasticNet' is not defined

In [116]:
elasticnet_r.coef_

NameError: name 'elasticnet_r' is not defined